""" import openpyxl 
from openpyxl import  load_workbook """

In [ ]:
import pandas as pd
import numpy as np
import schedule
import time
import os

#creamos la funcion MonitoringDueAcc()
def MonitoringDueAcc():
    try:
        #leemos el archivo de excel o csv almacenado dentro de una variable
        df = pd.read_csv("sample_data_intw.csv")        #encoding="utf-8", encoding_errors="replace"

        #convertir las fechas en formato datetime 
        df["last_rech_date_ma"] = pd.to_datetime(df["last_rech_date_ma"], errors="coerce")
        df["pdate"] = pd.to_datetime(df["pdate"], errors="coerce", dayfirst=True)

        #filtramos por la columna que tiene informacion requerida 
        deliquency_over30 = df[df["daily_decr30"] > 30]

        sol = deliquency_over30[[ "msisdn", "daily_decr30", "rental30",]].head(100)
        print(f"Data processed successfully.")
        return sol
    except Exception as e:
        print(f"An error has occurred: {e}")
        return pd.DataFrame()


def GenerateReportAuto():
    try:
        #el mensaje que se mostrara cuando se ejecute la funcion
        print("Report Generation In Process...") 
        sol = MonitoringDueAcc() #llamamos a la funcion MonitoringDueAcc()
        if not sol.empty: #si la variable sol no esta vacia
            #guardamos el archivo en formato excel
            output_file = os.path.join(os.getcwd(), "MonitoringDueAcc.xlsx") 
            sol.to_excel(output_file, index=False)
            #imprimimos el mensaje de que se genero el reporte exitosamente
            print(f"Report Generated Successfully at {output_file}") 
        else: #si la variable sol esta vacia mostrara el mensaje de que fallo la generacion del reporte
            print("Report Generation Failed.")
    except Exception as e: #si ocurre un error mostrara el siguient mensaje
        print(f"An error has occured: {e}")

#llamamos a la funcion GenerateReportAuto() para que se ejecute todos los viernes a las 8:00 AM
schedule.every().friday.at(f"08:00").do(GenerateReportAuto)
print(f"Waiting for the report to be Generated at 5:15 PM on Friday...")

#mantenemos el programa en ejecucion
while True:
    schedule.run_pending()
    time.sleep(2)




In [3]:
#Para contar a partir de una tabla frecuencia la cantidad de los numeros de telefono
df = pd.read_csv("sample_data_intw.csv")
frequency = df["msisdn"].value_counts().head(10)
frequency

msisdn
04581I85330    7
47819I90840    7
42825I88688    6
67324I84453    6
43430I70786    6
29191I82738    6
22038I88658    6
30080I90588    6
87592I84456    6
43096I88688    6
Name: count, dtype: int64

In [4]:
#para agrupar por msisdn y sumar los valores de daily_decr30
goup_by = df.groupby(["msisdn"])[["daily_decr90"]].sum().head(10)
goup_by

,daily_decr90
msisdn,
00004I82738,9.290
00005I96753,79.706
00010I89237,21070.120
00011I90846,20599.900
00012I70784,28595.720
00014I90843,39.096
00015I74510,16814.700
00015I82734,346.000
00015I97616,59.785


In [17]:
#para filtrar los valores del promedio de daily_decr30 mayores a 15
df["daily_decr30"] = np.where(df["daily_decr90"].mean() < 15, 1, 0)
df["daily_decr30"] 

0         0
1         0
2         0
3         0
4         0
         ..
209588    0
209589    0
209590    0
209591    0
209592    0
Name: daily_decr30, Length: 209593, dtype: int64

In [18]:
df["daily_decr90"].value_counts()

daily_decr90
0.000000       4063
500.000000      782
1000.000000     490
700.000000      343
600.000000      319
               ... 
14.078000         1
58.023333         1
291.563333        1
1238.560000       1
4534.820000       1
Name: count, Length: 155483, dtype: int64